## Verifying the linear algebra operations

In [1]:
import numpy as np
from string import ascii_lowercase

n_beads_with_symbols = {i:10 for i in ascii_lowercase}

# Representing the words as a matrix of counts of each letter in a word
words = ["abc", "def", "cba", "fed",]
count_letters_of_words = np.array([
            [word.count(letter) for letter in sorted(n_beads_with_symbols)]
            for word in words
        ])

count_letters_of_words

array([[1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0],
       [0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0],
       [1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0],
       [0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0]])

In [2]:
# Representing the solutions as a matrix of counts of each word
solutions = np.array([
    [1, 1, 0, 0],
    [0, 0, 0, 0],
    [1, 1, 1, 1],
    [0, 0, 0, 1],
    [1, 1, 0, 1]
])

# The dot product of the solutions and the words will give us the number of beads used in a solution
np.dot(solutions, count_letters_of_words)

array([[1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0],
       [2, 2, 2, 2, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0],
       [0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0],
       [1, 1, 1, 2, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0]])

In [3]:
# Checking if substraction works as i expect
beads = np.array([n_beads_with_symbols[letter] for letter in sorted(n_beads_with_symbols)])
beads - np.dot(solutions, count_letters_of_words)

array([[ 9,  9,  9,  9,  9,  9, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10,
        10, 10, 10, 10, 10, 10, 10, 10, 10, 10],
       [10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10,
        10, 10, 10, 10, 10, 10, 10, 10, 10, 10],
       [ 8,  8,  8,  8,  8,  8, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10,
        10, 10, 10, 10, 10, 10, 10, 10, 10, 10],
       [10, 10, 10,  9,  9,  9, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10,
        10, 10, 10, 10, 10, 10, 10, 10, 10, 10],
       [ 9,  9,  9,  8,  8,  8, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10,
        10, 10, 10, 10, 10, 10, 10, 10, 10, 10]])

In [4]:
np.sum(beads - np.dot(solutions, count_letters_of_words), axis=1)

array([254, 260, 248, 257, 251])

## The problem definition

In [5]:
import numpy as np
import pandas as pd
from pymoo.core.problem import Problem

class FriendshipBraceletProduction(Problem):
    def __init__(self, word_weights: list[tuple[str, float]], n_beads_with_symbols: dict[str, int], remember_data=True, use_G=True):
        words, weights = zip(*word_weights)
        self.words = np.array(words)
        self.weights = np.array(weights)
        self.use_G = use_G
        
        self.count_letters_of_words = np.array([
            [word.count(letter) for letter in sorted(n_beads_with_symbols)]
            for word in self.words
        ])
        self.beads = np.array([n_beads_with_symbols[letter] for letter in sorted(n_beads_with_symbols)])

        self.remember_data = remember_data
        if remember_data:
            self.turn = 0
            self._columns = [f"x_{w}" for w in words]+[f"leftover_{l}" for l in sorted(n_beads_with_symbols)]+["fitness"]
            self.data_dump = pd.DataFrame(columns=self._columns+['turn'])
        else:
            self.data_dump = None

        super().__init__(
            n_var=len(word_weights), n_obj=1, 
            n_ieq_constr=(len(n_beads_with_symbols) if use_G else 0),
            xl = 0, xu = 8, vtype=int
        )

    def _evaluate(self, x, out, *args, **kwargs):
        weight_sum = np.sum(self.weights * x, axis=1)
        unique_words = np.count_nonzero(x, axis=1)
        leftover_beads = self.beads - np.dot(x, self.count_letters_of_words)
        
        out["F"] = -weight_sum*3.0 - unique_words + np.sum(leftover_beads, axis=1)/26*0.2
        if self.use_G:
            out["G"] = -leftover_beads
        
        if self.remember_data:
            concated = np.hstack((
                x, leftover_beads, out["F"][:, None]
            ))
            dataframed = pd.DataFrame(concated, columns=self._columns)
            dataframed['turn'] = self.turn
            self.turn += 1
            self.data_dump = pd.concat((self.data_dump, dataframed), ignore_index=True)

In [6]:
from pymoo.algorithms.soo.nonconvex.ga import GA
from pymoo.operators.crossover.sbx import SBX
from pymoo.operators.mutation.pm import PM
from pymoo.operators.repair.rounding import RoundingRepair
from pymoo.operators.sampling.rnd import IntegerRandomSampling
from pymoo.termination.ftol import SingleObjectiveSpaceTermination
from pymoo.termination.robust import RobustTermination
from pymoo.optimize import minimize

def run_opt(weights, n_beads_with_symbols, pop_size=10, remember_data=True, algorithm=None):
    problem = FriendshipBraceletProduction(weights, n_beads_with_symbols, remember_data=remember_data)

    if algorithm is None:
        method = GA(pop_size=pop_size,
                    sampling=IntegerRandomSampling(),
                    crossover=SBX(repair=RoundingRepair()),
                    mutation=PM(repair=RoundingRepair()),
                    )
    else:
        method = algorithm

    res = minimize(
                    problem,
                    method,
                    termination=RobustTermination(SingleObjectiveSpaceTermination(tol=1e-9)),
                    save_history=True,
                    seed=22,
                    verbose=True
                )

    return res.X, res.F, res.CV, res.G, problem.data_dump

## Example

In [7]:
from string import ascii_lowercase

words = ["abc", "def", "cba", "fed"]
weights = [(i, 1.0) for i in words]
n_beads_with_symbols = {i:10 for i in ascii_lowercase}

X, F, CV, G, data_dump = run_opt(weights, n_beads_with_symbols)


Compiled modules for significant speedup can not be used!
https://pymoo.org/installation.html#installation

To disable this warning:
from pymoo.config import Config
Config.warnings['not_compiled'] = False

n_gen  |  n_eval  |     cv_min    |     cv_avg    |     f_avg     |     f_min    
     1 |       10 |  0.000000E+00 |  7.2000000000 | -3.727115E+01 | -4.734615E+01
     2 |       20 |  0.000000E+00 |  0.000000E+00 | -4.089769E+01 | -5.339231E+01
     3 |       30 |  0.000000E+00 |  0.000000E+00 | -4.613692E+01 | -5.339231E+01
     4 |       40 |  0.000000E+00 |  0.000000E+00 | -4.875769E+01 | -5.339231E+01
     5 |       50 |  0.000000E+00 |  0.000000E+00 | -5.026923E+01 | -5.943846E+01
     6 |       60 |  0.000000E+00 |  0.000000E+00 | -5.399692E+01 | -6.246154E+01
     7 |       70 |  0.000000E+00 |  0.000000E+00 | -5.581077E+01 | -6.246154E+01
     8 |       80 |  0.000000E+00 |  0.000000E+00 | -5.611308E+01 | -6.246154E+01
     9 |       90 |  0.000000E+00 |  0.000000E+00 | -5.

/tmp/ipykernel_77585/302386517.py:48: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  self.data_dump = pd.concat((self.data_dump, dataframed), ignore_index=True)


In [8]:
data_dump

,x_abc,x_def,x_cba,x_fed,leftover_a,leftover_b,leftover_c,leftover_d,leftover_e,leftover_f,...,leftover_s,leftover_t,leftover_u,leftover_v,leftover_w,leftover_x,leftover_y,leftover_z,fitness,turn
0,5.0,8.0,6.0,7.0,-1.0,-1.0,-1.0,-5.0,-5.0,-5.0,...,10.0,10.0,10.0,10.0,10.0,10.0,10.0,10.0,-80.600000,0
1,4.0,7.0,3.0,1.0,3.0,3.0,3.0,2.0,2.0,2.0,...,10.0,10.0,10.0,10.0,10.0,10.0,10.0,10.0,-47.346154,0
2,0.0,2.0,3.0,1.0,7.0,7.0,7.0,7.0,7.0,7.0,...,10.0,10.0,10.0,10.0,10.0,10.0,10.0,10.0,-19.138462,0
3,4.0,8.0,2.0,7.0,4.0,4.0,4.0,-5.0,-5.0,-5.0,...,10.0,10.0,10.0,10.0,10.0,10.0,10.0,10.0,-65.484615,0
4,6.0,8.0,7.0,3.0,-3.0,-3.0,-3.0,-1.0,-1.0,-1.0,...,10.0,10.0,10.0,10.0,10.0,10.0,10.0,10.0,-74.553846,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
355,4.0,4.0,7.0,4.0,-1.0,-1.0,-1.0,2.0,2.0,2.0,...,10.0,10.0,10.0,10.0,10.0,10.0,10.0,10.0,-59.438462,35
356,5.0,4.0,6.0,7.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,...,10.0,10.0,10.0,10.0,10.0,10.0,10.0,10.0,-68.507692,35
357,4.0,3.0,4.0,5.0,2.0,2.0,2.0,2.0,2.0,2.0,...,10.0,10.0,10.0,10.0,10.0,10.0,10.0,10.0,-50.369231,35
358,4.0,4.0,5.0,6.0,1.0,1.0,1.0,0.0,0.0,0.0,...,10.0,10.0,10.0,10.0,10.0,10.0,10.0,10.0,-59.438462,35


In [9]:
print("Best solution found:", X)
print("Function value:", -F)
print("G:", -G)
print("Constraints violation:", CV)

Best solution found: [4 5 6 5]
Function value: [62.46153846]
G: [-0. -0. -0. -0. -0. -0. 10. 10. 10. 10. 10. 10. 10. 10. 10. 10. 10. 10.
 10. 10. 10. 10. 10. 10. 10. 10.]
Constraints violation: [0.]


## Run on real data

### Data cleanup

In [10]:
from pandas import read_csv

df = read_csv("friendship-bracelets.csv", sep="\t")

df['words'] = df['words'] \
    .str.lower() \
    .str.replace(r"\s+", " ", regex=True) \
    .str.strip()
df['clean_words'] = df['words'] \
    .str.replace(r"[^a-z ]", "", regex=True) \
    .str.replace(r"\s+", " ", regex=True) \
    .str.strip()

df = df[df["clean_words"].str.len() > 1]
df.head()

,words,clean_words
0,taylor swift,taylor swift
1,debut,debut
2,fearless,fearless
3,reputation,reputation
4,speak now,speak now


In [11]:
# Create acronyms
acronyms = df['clean_words'].str.split().apply(lambda x: "".join([i[0] for i in x]))

# Create weight list
df.sort_values("words", inplace=True)
weights = [(i, 1.0) for i in df['clean_words']]
words = list(df['words'])

# Add acronyms to the weights
weights.extend([(i, 0.2) for i in acronyms if len(i) > 3])
words.extend(list(acronyms[acronyms.str.len() > 3]))
len(weights)


704

In [12]:
# Count letters to exclude the rarest ones

from collections import Counter

c = Counter("".join(i[0] for i in weights))
c_lowest = [i[0] for i in c.most_common()][-4:]
c.most_common()

[(' ', 1005),
 ('e', 798),
 ('t', 651),
 ('a', 602),
 ('i', 577),
 ('o', 526),
 ('s', 489),
 ('n', 438),
 ('r', 407),
 ('l', 362),
 ('h', 360),
 ('m', 308),
 ('d', 273),
 ('y', 254),
 ('c', 220),
 ('w', 206),
 ('u', 205),
 ('g', 196),
 ('b', 186),
 ('f', 151),
 ('p', 108),
 ('k', 105),
 ('v', 81),
 ('j', 21),
 ('z', 9),
 ('x', 6),
 ('q', 4)]

In [13]:
from string import ascii_lowercase
n_beads_with_symbols = {i:40 for i in ascii_lowercase if i not in c_lowest}

### Run

In [14]:
#X, F, CV, G, data_dump = run_opt(weights, n_beads_with_symbols, pop_size=500, remember_data=False)

In [15]:
def interpret_solution(X):
    global words
    return {words[i]:x for i, x in enumerate(X) if x > 0}
    # return {f"{words[i]} ({weights[i][0]})":x for i, x in enumerate(X) if x > 0}

def interpret_g(G):
    global n_beads_with_symbols
    return {list(n_beads_with_symbols.keys())[i]:-G[i] for i in range(len(n_beads_with_symbols)) if G[i] < 0}

In [16]:
# solution = interpret_solution(X)
# print(f"Best solution found ({sum(solution.values())} words):")
# for key, value in solution.items():
#     print(f"\t{key}: {value}")

# leftovers = interpret_g(G)
# print(f"Leftover letters:")
# for key, value in leftovers.items():
#     print(f"\t{key}: {value}")
# print(int(sum(leftovers.values())), "unused out of", sum(n_beads_with_symbols.values()), "total")
# print("Efficiency:", 
#       round(
#             1-sum(leftovers.values())/sum(n_beads_with_symbols.values()),
#             2
#         )
#     )
# print("Function value:", -F)
# print("Constraints violation:", CV)

# Hyperparameter optimization

In [17]:
from pymoo.algorithms.hyperparameters import HyperparameterProblem, MultiRun, stats_single_objective_mean
from pymoo.algorithms.soo.nonconvex.optuna import Optuna
from pymoo.core.parameters import set_params, hierarchical
from pymoo.optimize import minimize
from pymoo.problems.single import Sphere

problem = FriendshipBraceletProduction(weights, n_beads_with_symbols, remember_data=False, use_G=False)

algorithm = GA(
    pop_size=400,
    sampling=IntegerRandomSampling(),
    crossover=SBX(repair=RoundingRepair()),
    mutation=PM(repair=RoundingRepair()),
)

n_evals = 500
seeds = [22, 69]

performance = MultiRun(problem, seeds=seeds, func_stats=stats_single_objective_mean, termination=("n_evals", n_evals))

res = minimize(HyperparameterProblem(algorithm, performance),
               Optuna(),
               termination=RobustTermination(SingleObjectiveSpaceTermination(tol=1e-6)),
               seed=1,
               verbose=True)

hyperparams = res.X
print("Hypers:", hyperparams)
set_params(algorithm, hierarchical(hyperparams))


/home/jqdq/Documents/repos/friendship-optimizer/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


n_gen  |  n_eval  |     f_avg     |     f_min    
     1 |        1 | -8.705162E+03 | -8.705162E+03
     2 |        2 | -8.828050E+03 | -8.828050E+03
     3 |        3 | -8.825927E+03 | -8.828050E+03
     4 |        4 | -8.462600E+03 | -8.828050E+03
     5 |        5 | -9.259873E+03 | -9.259873E+03
     6 |        6 | -9.775027E+03 | -9.775027E+03
     7 |        7 | -8.801485E+03 | -9.775027E+03
     8 |        8 | -9.165265E+03 | -9.775027E+03
     9 |        9 | -9.068704E+03 | -9.775027E+03
    10 |       10 | -9.721996E+03 | -9.775027E+03
    11 |       11 | -9.772958E+03 | -9.775027E+03
    12 |       12 | -9.789242E+03 | -9.789242E+03
    13 |       13 | -9.616123E+03 | -9.789242E+03
    14 |       14 | -9.440646E+03 | -9.789242E+03
    15 |       15 | -9.656465E+03 | -9.789242E+03
    16 |       16 | -9.422338E+03 | -9.789242E+03
    17 |       17 | -9.651554E+03 | -9.789242E+03
    18 |       18 | -9.370065E+03 | -9.789242E+03
    19 |       19 | -9.510554E+03 | -9.789242E+03


In [18]:
X, F, CV, G, data_dump = run_opt(weights, n_beads_with_symbols, remember_data=False, algorithm=algorithm)

n_gen  |  n_eval  |     cv_min    |     cv_avg    |     f_avg     |     f_min    
     1 |      400 |  2.590900E+04 |  2.918528E+04 |             - |             -
     2 |      800 |  1.510500E+04 |  2.347469E+04 |             - |             -
     3 |     1200 |  5.776000E+03 |  1.420064E+04 |             - |             -
     4 |     1600 |  1.266000E+03 |  7.494240E+03 |             - |             -
     5 |     2000 |  1.0000000000 |  2.708290E+03 |             - |             -
     6 |     2400 |  0.000000E+00 |  4.937050E+02 | -6.317391E+01 | -1.227615E+02
     7 |     2800 |  0.000000E+00 |  1.510000E+01 | -4.023421E+01 | -1.227615E+02
     8 |     3200 |  0.000000E+00 |  0.000000E+00 | -5.343771E+01 | -1.307538E+02
     9 |     3600 |  0.000000E+00 |  0.000000E+00 | -7.569031E+01 | -1.307538E+02
    10 |     4000 |  0.000000E+00 |  0.000000E+00 | -8.875013E+01 | -1.357615E+02
    11 |     4400 |  0.000000E+00 |  0.000000E+00 | -9.818206E+01 | -1.424154E+02
    12 |     480

In [19]:
solution = interpret_solution(X)
print(f"Best solution found ({sum(solution.values())} words):")
for key, value in solution.items():
    print(f"\t{key}: {value}")

leftovers = interpret_g(G)
print(f"Leftover letters:")
for key, value in leftovers.items():
    print(f"\t{key}: {value}")
print(int(sum(leftovers.values())), "unused out of", sum(n_beads_with_symbols.values()), "total")
print("Efficiency:", 
      round(
            1-sum(leftovers.values())/sum(n_beads_with_symbols.values()),
            2
        )
    )
print("Function value:", -F)
print("Constraints violation:", CV)

Best solution found (129 words):
	2 am: 1
	betty: 1
	birch: 2
	cardigan: 1
	cassandra: 1
	clean: 1
	closure: 1
	clowning: 2
	daylight: 1
	dibbles: 1
	don't say yes: 1
	don't you: 1
	down bad: 1
	dress: 1
	eras: 1
	facts: 1
	fancy shit: 1
	florida: 5
	folkwhore: 1
	ghosts: 2
	glitch: 1
	gold rush: 1
	haunted: 1
	hoax: 1
	i heart ?: 1
	i'd lie: 1
	i'm nyc: 1
	idk: 2
	inez: 1
	ivy: 1
	james: 1
	karma: 1
	light me up: 1
	loml: 2
	lover: 1
	lwymmd: 1
	mad love: 1
	maroon: 1
	me!: 1
	my body: 2
	my hips: 2
	nice!: 1
	old pickup truck: 2
	ours: 2
	peace: 1
	peter: 1
	red: 2
	robin: 1
	ronan: 1
	run: 3
	rwylm: 5
	seven: 1
	slut!: 2
	speak now: 2
	style: 1
	superman: 2
	t swizzle: 1
	the 1: 1
	tiwwchnt: 1
	u fancy me: 4
	ur gay: 1
	wanegbt: 1
	willow: 1
	yoyok: 1
	ybwm: 1
	icsy: 1
	tvfn: 1
	wtny: 1
	hygtg: 1
	idsb: 2
	ithk: 1
	yntcd: 1
	nbnc: 1
	ttpd: 1
	bdilh: 1
	cwywm: 2
	iwyb: 1
	homh: 1
	cmcd: 1
	bycpd: 1
	ysnb: 1
	htmc: 1
	osns: 1
	dcmk: 1
	dcmb: 1
	fith: 1
	bftw: 2
	bbmb: 2
	wbfb: 4
	mtfb